In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Pobranie danych
from ucimlrepo import fetch_ucirepo 
ds = fetch_ucirepo(id=23)
df = pd.concat([ds.data.features, ds.data.targets], axis=1)

In [ ]:
# Zmiana opisu odległości od mata na liczby
mapping = {
    "draw": -1,
    "zero": 0,
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "ten": 10,
    "eleven": 11,
    "twelve": 12,
    "thirteen": 13,
    "fourteen": 14,
    "fifteen": 15,
    "sixteen": 16
}
df["white-depth-of-win"] = df["white-depth-of-win"].map(mapping)

In [ ]:
# Zmiana nazw pól z literek na liczby
mapping_files = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": 4,
    "e": 5,
    "f": 6,
    "g": 7,
    "h": 8
}
df["white-king-file"] = df["white-king-file"].map(mapping_files)
df["white-rook-file"] = df["white-rook-file"].map(mapping_files)
df["black-king-file"] = df["black-king-file"].map(mapping_files)

In [ ]:
# Kolumny metryk
dx = df["white-king-file"] - df["black-king-file"]
dy = df["white-king-rank"] - df["black-king-rank"]

df["euclides"] = np.sqrt(dx**2 + dy**2)
df["manhattan"] = np.abs(dx) + np.abs(dy)
df["czebyszew"] = np.maximum(np.abs(dx), np.abs(dy))

In [ ]:
# Odległość czarnego króla od krawędzi
df["black_edge"] = np.minimum.reduce([
    df["black-king-file"] - 1,
    8 - df["black-king-file"],
    df["black-king-rank"] - 1,
    8 - df["black-king-rank"]
])

In [ ]:
# Podczas mata czarny król zawsze znajduje się przy krawędzi planszy, najczęściej w rogu A1.
# W przypadku remisu czarny król znacznie częściej stoi dalej od rogu i bardziej w środku planszy, 
# najczęściej w rejonie kolumn F–G.

# mat
mate = df[df["white-depth-of-win"] == 0]

heat_mate = pd.crosstab(
    mate["black-king-rank"],
    mate["black-king-file"]
)

heat_mate = heat_mate.reindex(index=range(1, 9), columns=range(1, 9), fill_value=0)
heat_mate = heat_mate.sort_index(ascending=False)

# pat
draws = df[df["white-depth-of-win"] == -1]

heat_draw = pd.crosstab(
    draws["black-king-rank"],
    draws["black-king-file"]
)
heat_draw = heat_draw.reindex(index=range(1, 9), columns=range(1, 9), fill_value=0)
heat_draw = heat_draw.sort_index(ascending=False)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(heat_mate, interpolation="nearest", cmap="Blues")
plt.title("Czarny król podczas mata")
plt.xticks(range(8), list("ABCDEFGH"))
plt.yticks(range(8), range(8, 0, -1))
plt.colorbar(label="liczba wystąpień")

plt.subplot(1, 2, 2)
plt.imshow(heat_draw, interpolation="nearest", cmap="Blues")
plt.title("Czarny król podczas pata")
plt.xticks(range(8), list("ABCDEFGH"))
plt.yticks(range(8), range(8, 0, -1))
plt.colorbar(label="liczba wystąpień")

plt.tight_layout()
plt.show()

In [ ]:
# Wraz ze wzrostem dystansu pomiędzy białym i czarnym królem rośnie średnia liczba ruchów do zakończenia 
# gry, co sugeruje że odległość król–król ma wpływ na tempo mata. Najbardziej regularną 
# zależność daje metryka Manhattan, dlatego w tych danych jest najlepsza do szacowania liczby ruchów.

# ignorujemy remisy
df_nodraw = df[df["white-depth-of-win"] != -1].copy()

avg_manhattan = df_nodraw.groupby("manhattan")["white-depth-of-win"].mean()
avg_euclides  = df_nodraw.groupby("euclides")["white-depth-of-win"].mean()
avg_czebyszew = df_nodraw.groupby("czebyszew")["white-depth-of-win"].mean()

plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.plot(avg_euclides.index, avg_euclides.values, marker="o")
plt.title("Średnia liczba ruchów vs Euklides")
plt.xlabel("euclides")
plt.ylabel("średni white-depth-of-win")
plt.grid(True)

plt.subplot(1,3,2)
plt.plot(avg_manhattan.index, avg_manhattan.values, marker="o")
plt.title("Średnia liczba ruchów vs Manhattan")
plt.xlabel("manhattan")
plt.ylabel("średni white-depth-of-win")
plt.grid(True)

plt.subplot(1,3,3)
plt.plot(avg_czebyszew.index, avg_czebyszew.values, marker="o")
plt.title("Średnia liczba ruchów vs Czebyszew")
plt.xlabel("czebyszew")
plt.ylabel("średni white-depth-of-win")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Odległość czarnego króla od krawędzi planszy jest bardzo przydatna do przewidywania liczby ruchów do
# mata. Widzimy, że im dalej czarny król znajduje się od krawędzi, tym średnio dłużej trwa 
# zakończenie partii. Dodatkowo uwzględnienie dystansu między królami (wybrałem metrykę Manhattan bo była najlepsza)
# jeszcze poprawia oszacowanie dla tej samej odległości od krawędzi pozycje z większym dystansem król–król 
# wymagają średnio większej liczby ruchów.

# wykres zależności liczby ruchów do mata, a odległością czarnego króla od krawędzi
df_nodraw = df[df["white-depth-of-win"] != -1].copy()

# wykres 1 
avg_edge = df_nodraw.groupby("black_edge")["white-depth-of-win"].mean()

# wykres 2 - to samo ale jeszcze z podziałem na odległość Manhattan
df_nodraw["manhattan_group"] = pd.qcut(
    df_nodraw["manhattan"], q=3, labels=["blisko", "średnio", "daleko"]
)

pivot = df_nodraw.pivot_table(
    values="white-depth-of-win",
    index="black_edge",
    columns="manhattan_group",
    aggfunc="mean",
    observed=True
)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(avg_edge.index, avg_edge.values, marker="o")
plt.title("Wpływ odległości czarnego króla krawędzi na liczbę ruchów do mata")
plt.xlabel("Odległość czarnego króla od krawędzi")
plt.ylabel("Średnia liczba ruchów do mata")
plt.grid(True)

plt.subplot(1, 2, 2)
blue_shades = ["#9ecae1", "#3182bd", "#08519c"]

for (col, color) in zip(pivot.columns, blue_shades):
    plt.plot(pivot.index, pivot[col].values, marker="o", label=col, color=color)

plt.title("Wpływ odległości czarnego króla krawędzi i odległość między królami\n w metryce Manhattan na liczbę ruchów do mata")
plt.xlabel("Odległość czarnego króla od krawędzi")
plt.ylabel("Średnia liczba ruchów do mata")
plt.grid(True)
plt.legend(title="Dystans między królami")

plt.tight_layout()
plt.show()